<h1>Deep Learning</h1>
<li>Rough idea: mimic the human brain (not that we know how that works!)
<li>Neural networks: learn through examples with no procedural learning algorithm
<li>As wikipedia says: <span style="color:blue">vaguely</span> inspired by animal brains

<h2>Structure of neural networks</h2>
<li>A neural network is a directed often acyclic graph
<li><span style="color:red">Neurons</span>: nodes
<li><span style="color:red">Synapses</span>: connections (edges) between nodes that contain weights
<li>a neuron calculates a weighted sum of its input nodes and then decides whether to fire or not (should it react to the input or not)
<li><span style="color:red">Activation functions</span>: the function that makes the activation decision
<li><span style="color:red">Layers</span>: aggregations of neurons that use the same transformation function (different layers can use different functions)
<li><span style="color:red">Input layer</span>: feature values from example cases enter the network here (think of it as the sensory organ of the network)
<li><span style="color:red">Output layer</span>: the result (action) layer (classes, continuous values)
<li><span style="color:red">Hidden layers</span>: all the layers between the input and output layers

In [ ]:
from IPython.display import Image
Image(filename="neuron.png",height=500,width=500)

<h2>Example of a network</h2>
<small>from: <i>https://upload.wikimedia.org/wikipedia/commons/4/46/Colored_neural_network.svg</i></small>

In [ ]:
from IPython.display import Image
Image(url="https://upload.wikimedia.org/wikipedia/commons/4/46/Colored_neural_network.svg")

<h2>Rough procedure: "for dummies"!</h2>
<li>INITIALIZE: randomly assign weights to each connection
<li>RUN:
<ol>
<li>give an example to the network
<li>calculate the weighted sum of inputs at each neuron
<li>apply the activation function to that weighted sum
<li>do this layer by layer until you get the output layer values
<li>calculate the difference between calculated values and actual values
<li>tweak weights in the entire network so that the calculate output gets "marginally" closer to the actual value
<li>rinse and repeat

<h1>A super simple example</h1>
<li>Each input case has 3 features, each of which is either a 0 or a 1
<li>We need to classify each input case into either a 0 or 1 (binary classification)
<li>The actual "real world" rule is that <i>if feature 1 is 0, the case is classified as 0. If feature 1 is 1, the case is classified as 1</i></li>
<li>(In other words, features 2 and 3 contain no information)</li>
<li>We want the net to learn this

In [ ]:
from IPython.display import Image
Image(filename="simple_network.png",height=500,width=500)

<h3>Define inputs and outputs</h3>

In [ ]:
import numpy as np 


In [ ]:
X = np.array([  [0,0,1],
                [0,1,1],
                [1,0,1],
                [1,1,1] ])
y = np.array([[0,0,1,1]]).T

<h3>Initialization</h3>
<li>generate random weights for every edge in the network
<li>we'll use np.random.random (generates random numbers between 0 and 1)
<li>and adjust the weights so that they are between -1 and 1

In [ ]:
np.random.seed(1)
syn0 = 2*np.random.random((3,1)) - 1
syn0

<h2>Define the activation function</h2>


<li>Let's see what our weighted sums look like

In [ ]:
np.dot(X,syn0)

<li>Nice. We could say that anything less than 0 is a 0 and anything greater than 0 is a 1
<li>And use these "predicted" 0s and 1s to compute the error
<li>And use these errors to adjust weights
<li>Not ideal, because:
<ul>
<li>learning would be binary and the model would keep switching from class 0 to class 1 
<li>what we would like is for learning to be smooth
<li>"hmm, looks like a class 1 but i'll just tweak the probability that it is a class 1 rather than switch entirely to a class 1"
<li>"that way, over time, I'll get more and more sure"

<h2>Sigmoid function</h2>
<small>https://upload.wikimedia.org/wikipedia/commons/8/88/Logistic-curve.svg</small>


In [ ]:
from IPython.display import Image
Image(url="https://upload.wikimedia.org/wikipedia/commons/8/88/Logistic-curve.svg")


The sigmoid function takes a number as its argument and returns a value between 0 and 1. It has the following properties:<br>
$ sigmoid(x) = \dfrac{1}{1+e^{-x}} $
<li>sigmoid(0) returns 0.5</li>
<li>the derivative (slope) of the function is higher as values approach 0 and lower as values move away from 0</li>
<li>the derivative is easy to calculate because it is a function of the sigmoid value</li>
$ \dfrac{d}{dx}sigmoid(x) = sigmoid(x)*(1-sigmoid(x)) $


<h3>Only if you're interested in deriving the derivative</h3>
The quotient rule for derivates is<p>
$ \frac{d }{dx}\frac{f(x)}{g(x)} = \dfrac{(g(x)\times\frac{d }{dx}f(x)) -(f(x)\times\frac{d}{dx}g(x)))}{g(x)^2} $
    </p>
    Therefore: <p>
$ \dfrac{d}{dx}sigmoid(x) $ <br>
    $ = \dfrac{(1+e^{-x})*(0) - (1)(-e^{-x})}{(1+e^{-x})^2} $<br>
    $ = \dfrac{ - (1)(-e^{-x})}{(1+e^{-x})^2} $<br>
    $ = \dfrac{e^{-x}}{(1+e^{-x})^2} $<br>
    $ = \dfrac{1 + e^{-x} - 1}{(1+e^{-x})^2} $<br>
    $ = \dfrac{1 + e^{-x}}{(1+e^{-x})^2} - \dfrac{1}{(1+e^{-x})^2} $<br>
    $ = \dfrac{1}{(1+e^{-x})} - \dfrac{1}{(1+e^{-x})^2} $<br>
    $ = \dfrac{1}{(1+e^{-x})}\times(1-\dfrac{1}{(1+e^{-x})}) $ <br>
    $ = sigmoid(x)*(1-sigmoid(x)) $
    </p>
    

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
print(sigmoid(5.5) - sigmoid(5.45))
print(sigmoid(1.0) - sigmoid(0.95))
print(sigmoid(.5) - sigmoid(.45))
print(sigmoid(0.05) - sigmoid(0.0))



<h2>Now let's train the network</h2>
<li>First we'll multiply X by the weight array
<li>The linear combination we talked about
<li>Then apply the sigmoid function to introduce nonlinearity and convert the total to a (0,1) range
<li>level_0: First (input layer)
<li>level_1: Second (hidden layer - output layer in our case)
<li>Then compute the error (y - level_1)
<li>Finally adjust the weights
<ul>
<li>The sigmoid function gives us the rate at which we want to move the weights
<li>multiply the error by the slope (derivative) at that point
<li>the slope at point x is d/dx sigmoid(x) = sigmoid(x)*(1-sigmoid(x))
<li>notice that the slope is lower at the extremes and higher in the middle!
<li>Use these deltas to adjust the weights
</ul>
<li>Repeat with the next set of training cases

In [ ]:
X,y

In [ ]:
def sigmoid(x,deriv=False):
    if deriv:
        return sigmoid(x)*(1-sigmoid(x))
    return 1/(1+np.exp(-x))

print(sigmoid(-1),sigmoid(-1,True))

In [ ]:
def sigmoid(x,deriv=False):
    if deriv:
        return sigmoid(x)*(1-sigmoid(x))
    return 1/(1+np.exp(-x))

def run_net(X,y,activation_function=sigmoid,passes=10):
    np.random.seed(1) #seed the random numbers
    syn0 = 2*np.random.random((3,1)) - 1    #Calculate initial weights
    for i in range(0,passes):
        level_0 = X  #Input to the nn 
        level_1 = activation_function(np.dot(level_0,syn0)) #New weights 
        level_1_error = y - level_1 #error (note: y is 1/0; level_1 is in the range (0,1))
        #Get the derivative of the sigmoid (the change) and multiply by the error
        level_1_delta = level_1_error * activation_function(level_1,True)
        syn0 += np.dot(level_0.T,level_1_delta) #Update the weights (level_0 * deltas)
    return syn0

In [ ]:
final_weights = run_net(X,y,sigmoid,10)
final_weights

In [ ]:
test_X = np.array([[1,1,1],[0,1,1],[1,0,0],[0,0,1]])
sigmoid(np.dot(test_X,final_weights))

<h2>Walkthrough</h2>

In [ ]:
level_0 = X #Input layer
syn0 = 2*np.random.random((3,1)) - 1 #initial weights
level_1 = sigmoid(np.dot(level_0,syn0)) #predicted y's
level_1_error = y - level_1 #error
level_1_delta = level_1_error * sigmoid(level_1,True) #change factor
syn0 += np.dot(level_0.T,level_1_delta) #new weights
print("level_0")
print(level_0,"\n")
print("syn0")
print(syn0,"\n")
print("level_1")
print(level_1,"\n")
print("y")
print(y,"\n")
print("level_1_error")
print(level_1_error,"\n")
print("deriv")
print(sigmoid(level_1,True),"\n")
print("level_1_delta")
print(level_1_delta,"\n")
print("new weights")
print(syn0,"\n")


<h2>Great! Let's try a different input</h2>
<li>If any two  or three are 1, then the class is 1
<li>Otherwise the class is zero

In [ ]:
import numpy as np
X = np.array([[0,0,1],
            [0,1,1],
            [1,0,1],
            [1,1,1],
             [1,1,0],
             [0,1,0],
             [1,0,0],
             [1,0,0]])
                
y = np.array([[0],[1],[1],[1],[1],[0],[0],[0]])

final_weights = run_net(X,y,sigmoid,10000)
test_X = np.array([[1,1,1],[0,1,1],[1,0,0],[0,0,1]])
sigmoid(np.dot(test_X,final_weights))

<h2>Not so good this time~</h2>

<li>The pattern here is non-linear</li>
<li>Nonlinearities can be captured by adding layers to the network</li>


<h2>Three layer network</h2>
<li>Input
<li>Hidden
<li>Output
<li>We need two sets of weights 
<li>Input layer has 3 nodes
<li>Hidden layer has 4 nodes
<li>Output layer has 1 node

In [ ]:
Image(filename="multi layer network.png",height=500,width=500)

<h2>Initialize</h2>
<li>randomly assign weights at each level

In [ ]:
syn0 = 2*np.random.random((3,4)) - 1
syn1 = 2*np.random.random((4,1)) - 1


In [ ]:
syn1

<h3>feed forward network</h3>
<li>Calculate node outputs at one level</li>
<li>Propagate changes to the next level</li>
<li>Information (input data) moves in one direction from the input layer to the output layer</li>
<li>There are no cycles</li>

In [ ]:
level_0 = X
level_1 = sigmoid(np.dot(level_0,syn0))
level_2 = sigmoid(np.dot(level_1,syn1))
level_2

<h2>Backpropagation</h2>
<li><b>backpropagation</b> refers to the algorithm used to adjust weights after the error term is computed</li>
<li>The error is propagated back from the output layer to the input layer one layer at a time, adjusting weights along the way</li>


In [ ]:
level_2_error = y - level_2
level_2_error

In [ ]:
level_2_delta = level_2_error*sigmoid(level_2,deriv=True)
level_2_delta

<li>Next, propagate the deltas back toward the input layer

In [ ]:
level_1_error = level_2_delta.dot(syn1.T)
level_1_delta = level_1_error * sigmoid(level_1,deriv=True)

<li>Calculate the new weights

In [ ]:
syn1 += level_1.T.dot(level_2_delta)
syn0 += level_0.T.dot(level_1_delta)

<h2>Putting it all together</h2>

In [ ]:
def sigmoid(x,deriv=False):
    if deriv:
        return x*(1-x)
    return 1/(1+np.exp(-x))

def run_net(X,y,activation_function=sigmoid,passes=10):
    import time
    np.random.seed(1)
    syn0 = 2*np.random.random((3,4)) - 1
    syn1 = 2*np.random.random((4,1)) - 1

    for i in range(passes):
        level_0 = X
        level_1 = activation_function(np.dot(level_0,syn0))
        level_2 = activation_function(np.dot(level_1,syn1))

        level_2_error = y - level_2

        level_2_delta = level_2_error*activation_function(level_2,deriv=True)

        level_1_error = level_2_delta.dot(syn1.T)

        level_1_delta = level_1_error * activation_function(level_1,deriv=True)

        syn1 += level_1.T.dot(level_2_delta)
        syn0 += level_0.T.dot(level_1_delta)
    print(level_2)
    return syn0,syn1

In [ ]:
syn0,syn1 = run_net(X,y,activation_function=sigmoid,passes=100)


<h3>Applying the net to test cases</h3>


In [ ]:
test_X

In [ ]:
level_0 = test_X
level_1 = sigmoid(np.dot(level_0,syn0))
level_2 = sigmoid(np.dot(level_1,syn1))
level_2